In [1]:
# run sentiment analysis for OpenAI API and Azure API

from utils import *
from tqdm import tqdm
import pandas as pd 
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI,AzureOpenAI

def inference(prompt, client, model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}], 
            max_tokens=4,
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print("!!!!! Error when processing prompt !!!!!")
        print(prompt)
        # print(f"!!!!! {e} !!!!!")
        return None

def batch_inference(batch_data, client, model, max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = list(
            tqdm(
                executor.map(lambda x: inference(x, client, model), batch_data),
                total=len(batch_data)
            )
        )
        for future in futures:
            results.append(future)
    return results

In [2]:
# dataset="FIQASA"
dataset="FPB"
test_dataset = TestDataset(dataset=dataset)

print("----- Load Dataset -----")
test_dataset = TestDataset(dataset=dataset)
batch_data = [test_dataset[i]['sentence'] for i in range(len(test_dataset))]

print("----- Inference Deepseek -----")
deepseek = OpenAI(api_key="sk-619b8461a6e546d5a69c207e9b37ddc2", base_url="https://api.deepseek.com")
deepseek_outputs = batch_inference(batch_data, deepseek, "deepseek-chat", 16)
save_to_csv(dataset_map[dataset]['path'], "deepseek", deepseek_outputs)

print("----- Inference Azure GPT-4O-Mini -----")
gpt = AzureOpenAI(api_key="b9135a15c242432cb20ddc43fea3a413", api_version="2023-06-01-preview", azure_endpoint="https://openai-oe.openai.azure.com/")
gpt_outputs = batch_inference(batch_data, gpt, "gpt-4o-mini", 16)
save_to_csv(dataset_map[dataset]['path'], "openai", gpt_outputs)

----- Load Dataset -----
----- Inference Deepseek -----


100%|██████████| 2264/2264 [02:33<00:00, 14.71it/s]


Save to data/sentimental_analysis/FPB/test.csv
----- Inference Azure GPT-4O-Mini -----


100%|██████████| 2264/2264 [01:16<00:00, 29.47it/s]

Save to data/sentimental_analysis/FPB/test.csv
